<h1> Import Nacessary Libary

In [ ]:
!pip install imblearn

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

<h1>Load Data from CSV file

In [ ]:
train_df=pd.read_csv("fraudTrain.csv")
train_df.head(5)

In [ ]:
test_df=pd.read_csv("fraudTest.csv")
test_df.head(4)

In [ ]:
combined_df=pd.concat([train_df, test_df], ignore_index=True)
data=combined_df

<h1>Class Labels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.countplot(x='is_fraud',color='purple', data=data)
plt.title('HISTOGRAM')
plt.xlabel('Fraud')
plt.ylabel('number')
plt.show()

<h2>Data Preprocessing

In [ ]:
data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
data['hour'] = data['trans_date_trans_time'].dt.hour
data['day_of_week'] = data['trans_date_trans_time'].dt.dayofweek
data['month'] = data['trans_date_trans_time'].dt.month
data = data.drop(['Unnamed: 0', 'trans_date_trans_time', 'merchant', 'first', 'last', 'street', 'city', 'state', 'zip', 'job', 'dob', 'trans_num'], axis=1)
data = pd.get_dummies(data, columns=['category'], drop_first=True)


data['gender'] = data['gender'].map({'M': 0, 'F': 1})


X = data.drop(['is_fraud'], axis=1)
y = data['is_fraud']
scaler = StandardScaler()
numerical_columns = ['amt', 'lat', 'long', 'city_pop', 'unix_time', 'merch_lat', 'merch_long']
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])

<h3>Training of Model

In [ ]:

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


smote = SMOTE(sampling_strategy=0.5, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Initialize and train Random Forest classifier with parallel processing
rf_classifier = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
rf_classifier.fit(X_train_resampled, y_train_resampled)


# Make predictions on the validation set
y_pred = rf_classifier.predict(X_val)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_val, y_pred))

print("ROC-AUC Score:", roc_auc_score(y_val, y_pred))



In [ ]:
from collections import Counter
print("Original Class Distribution:")
print(Counter(y_train))
print("Resampled Class Distribution:")
print(Counter(y_train_resampled))


In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

original_distribution = Counter(y_train)
resampled_distribution = Counter(y_train_resampled)
class_labels = list(original_distribution.keys())


original_counts = [original_distribution[label] for label in class_labels]
resampled_counts = [resampled_distribution[label] for label in class_labels]

plt.figure(figsize=(10, 6))
bar_width = 0.4
index = range(len(class_labels))

plt.bar(index, original_counts, width=bar_width, color='b', label='Original', align='center')
plt.bar(index, resampled_counts, width=bar_width, color='r', label='Resampled', alpha=0.7, align='edge')

plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution Before and After SMOTE')
plt.xticks(index, class_labels)
plt.legend()
plt.tight_layout()


for i, count in enumerate(original_counts):
    plt.text(i, count + 20, str(count), ha='center', va='bottom', fontsize=10, color='black')

for i, count in enumerate(resampled_counts):
    plt.text(i + bar_width, count + 20, str(count), ha='center', va='bottom', fontsize=10, color='black')

plt.show()

In [ ]:
plt.figure(figsize=(32, 20))
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()


In [ ]:
feature_importances = rf_classifier.feature_importances_
sorted_idx = feature_importances.argsort()

plt.figure(figsize=(10, 8))
plt.barh(range(X_train.shape[1]), feature_importances[sorted_idx], align='center')
plt.yticks(range(X_train.shape[1]), X_train.columns[sorted_idx])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importance Plot')
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()
